In [53]:
import pybaseball as pbb
import pandas as pd
import sklearn
import plotly as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

This is a test.

In [2]:
kershdata = pd.read_csv(r'c:\Users\Caleb\Documents\SchoolSSD\Data\ClaytonKershaw2023.csv')
kershdata.head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,0,CH,2023-09-30,84.4,1.56,6.29,"Kershaw, Clayton",527038,477132,fielders_choice,...,1,1,2,2,1,Standard,Strategic,156.0,0.105,1.087
1,1,FF,2023-09-30,89.2,1.76,6.24,"Kershaw, Clayton",527038,477132,NaN,...,1,1,1,1,1,Standard,Strategic,174.0,0.000,-0.115
2,2,SL,2023-09-30,86.3,1.67,6.23,"Kershaw, Clayton",527038,477132,NaN,...,1,1,1,1,1,Standard,Strategic,205.0,0.000,-0.078
3,3,CU,2023-09-30,73.1,1.40,6.46,"Kershaw, Clayton",642731,477132,force_out,...,1,1,1,1,1,Standard,Strategic,324.0,-0.022,-0.444
4,4,SL,2023-09-30,87.1,1.71,6.19,"Kershaw, Clayton",642731,477132,NaN,...,1,1,1,1,1,Standard,Strategic,208.0,0.000,0.051


Preprocessing below

In [63]:
kclean = kershdata[['pitch_type', 'balls', 'strikes', 'outs_when_up', 'events']]
kclean
encoder = OneHotEncoder()
kencode = encoder.fit_transform(kclean['pitch_type'].values.reshape(-1, 1)).toarray()
kencode2 = encoder.fit(kclean['pitch_type'].values.reshape(-1, 1))

RandomForest. Does not perform well..

In [66]:
X = kclean[['balls', 'strikes', 'outs_when_up']]
y = kencode
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = model.score(X_test, y_test)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.3645320197044335


Naive Bayes (Broken)

In [64]:
X = kclean[['balls', 'strikes', 'outs_when_up']]
y = kencode
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
nbmodel = MultinomialNB()
nbmodel.fit(X_train, y_train)
y_pred = nbmodel.predict(X_test)

accuracy = nbmodel.score(X_test, y_test)
print("Model Accuracy:", accuracy)

ValueError: y should be a 1d array, got an array of shape (1620, 7) instead.

Multinomial Logistic Regression


In [65]:
X = kclean[['balls', 'strikes', 'outs_when_up']]
y = kencode
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
LRmodel = LogisticRegression(multi_class='multinomial')
LRmodel.fit(X_train, y_train)
y_pred = LRmodel.predict(X_test)

accuracy = LRmodel.score(X_test, y_test)
print("Model Accuracy:", accuracy)

ValueError: y should be a 1d array, got an array of shape (1620, 7) instead.